In [ ]:
import pandas as pds
import numpy as np
import os
from IPython.display import display

In [ ]:
# Read and join tables
dir = os.getcwd()

leaderboards = pds.read_csv(os.path.join(dir,"data\\v1\\leaderboards-data.csv"), parse_dates=['date', 'verifiedDate'])
games = pds.read_csv(os.path.join(dir,"data\\v1\\games-data.csv"), parse_dates=['releaseDate', 'createdDate'])
users = pds.read_csv(os.path.join(dir,"data\\v1\\users-data.csv"), parse_dates=['signupDate'])

data_frame = leaderboards.merge(games, left_on="gameID", right_on="ID", how="left")
data_frame = data_frame.merge(users, left_on="players", right_on="ID", how="left")

for col in data_frame.columns[data_frame.dtypes == 'object']:
    data_frame[col] = data_frame[col].astype('category')

target = data_frame['emulated']
data_frame = data_frame.drop(columns=['runID', 'ID_x'])
data_frame = data_frame.rename(columns={'name_x': 'gameName', 'URL': 'gameURL', 'ID_y': 'userID', 'name_y': 'username'})

In [ ]:
# Print Table

pds.set_option('display.max_columns', None)
display(data_frame)
print(data_frame.dtypes)

In [ ]:
# Preprocessing

def replaceNaNsWithMostFrequent(value):
    return value.fillna(value.mode()[0])

def replaceNaNsWithMean(value):
    return value.fillna(value.mean())

def mapValuesToIntegers(value):
    return value.cat.codes

def convertToUNIXTimestamp(date):
    return (date.fillna("1970-01-01").astype('int64') // (10**9)).replace(0, np.NaN)

data_frame[['date', 
            'verifiedDate', 
            'releaseDate', 
            'createdDate', 
            'signupDate']] = data_frame[['date', 
                                         'verifiedDate', 
                                         'releaseDate', 
                                         'createdDate', 
                                         'signupDate']].apply(convertToUNIXTimestamp)
data_frame[['date', 
            'verifiedDate', 
            'releaseDate', 
            'createdDate', 
            'signupDate']] = data_frame[['date', 
                                         'verifiedDate', 
                                         'releaseDate', 
                                         'createdDate', 
                                         'signupDate']].apply(replaceNaNsWithMean)

for col in data_frame.columns[data_frame.dtypes == 'category']:
    data_frame[[col]] = data_frame[[col]].apply(replaceNaNsWithMostFrequent)
    data_frame[[col]] = data_frame[[col]].apply(mapValuesToIntegers)

display(data_frame)
print(data_frame.dtypes)